<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
!pip install tqdm --user

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [3]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [4]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [5]:
for i in range(1, 11):
    train_df['site%s' % i] = train_df['site%s' % i].map(sites_dict_df['site'])
    test_df['site%s' % i] = test_df['site%s' % i].map(sites_dict_df['site'])

In [6]:
train_df[sites].head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,safebrowsing.clients.google.com,safebrowsing-cache.google.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54843,safebrowsing.clients.google.com,safebrowsing-cache.google.com,safebrowsing.clients.google.com,safebrowsing-cache.google.com,NaN,NaN,NaN,NaN,NaN,NaN
77292,www.apache.org,www.apache.org,download.eclipse.org,www.apache.org,www.apache.org,www.webtide.com,download.oracle.com,javadl-esd-secure.oracle.com,www.caucho.com,www.apache.org
114021,www.webtide.com,download.oracle.com,www.caucho.com,download.oracle.com,www.webtide.com,www.apache.org,public.dhe.ibm.com,www.webtide.com,www.apache.org,www.apache.org
146670,public.dhe.ibm.com,jope.ow2.org,download.oracle.com,public.dhe.ibm.com,jope.ow2.org,master.dl.sourceforge.net,www.apache.org,download.eclipse.org,www.apache.org,public.dhe.ibm.com


In [7]:
 test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,www.facebook.com,2014-10-04 11:19:53,s-static.ak.facebook.com,2014-10-04 11:19:53,apis.google.com,2014-10-04 11:19:54,c1.adform.net,2014-10-04 11:19:54,www.google.com,2014-10-04 11:19:54,mpp2.vindicosuite.com,2014-10-04 11:19:54,api.nanigans.com,2014-10-04 11:19:54,www.google.fr,2014-10-04 11:19:54,secure.vistaprint.fr,2014-10-04 11:20:00,www.vistaprint.fr,2014-10-04 11:20:00
2,annotathon.org,2014-07-03 11:00:28,annotathon.org,2014-07-03 11:00:53,annotathon.org,2014-07-03 11:00:58,annotathon.org,2014-07-03 11:01:06,annotathon.org,2014-07-03 11:01:09,annotathon.org,2014-07-03 11:01:10,annotathon.org,2014-07-03 11:01:23,annotathon.org,2014-07-03 11:01:29,annotathon.org,2014-07-03 11:01:30,annotathon.org,2014-07-03 11:01:53
3,safebrowsing-cache.google.com,2014-12-05 15:55:12,safebrowsing-cache.google.com,2014-12-05 15:55:13,safebrowsing-cache.google.com,2014-12-05 15:55:14,safebrowsing-cache.google.com,2014-12-05 15:56:15,safebrowsing-cache.google.com,2014-12-05 15:56:16,safebrowsing-cache.google.com,2014-12-05 15:56:17,safebrowsing-cache.google.com,2014-12-05 15:56:18,safebrowsing-cache.google.com,2014-12-05 15:56:19,o2i.univ-bpclermont.fr,2014-12-05 15:56:33,o2i.univ-bpclermont.fr,2014-12-05 15:56:36
4,fr.yahoo.com,2014-11-04 10:03:19,yahoo.fr,2014-11-04 10:03:19,ocsp.verisign.com,2014-11-04 10:03:20,ocsp.thawte.com,2014-11-04 10:03:21,rapidssl-ocsp.geotrust.com,2014-11-04 10:03:21,clicks.beap.bc.yahoo.com,2014-11-04 10:03:22,ocsp.verisign.com,2014-11-04 10:03:22,ocsp.digicert.com,2014-11-04 10:03:22,ocsp.digicert.com,2014-11-04 10:03:23,clicks.beap.bc.yahoo.com,2014-11-04 10:03:23
5,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:31,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:32,sci.sciences.univ-bpclermont.fr,2014-05-16 15:05:33,go.microsoft.com,2014-05-16 15:05:39,windows.microsoft.com,2014-05-16 15:05:40,res2.windows.microsoft.com,2014-05-16 15:05:40,res1.windows.microsoft.com,2014-05-16 15:05:40,ajax.microsoft.com,2014-05-16 15:05:40,js.microsoft.com,2014-05-16 15:05:40,www.bing.com,2014-05-16 15:05:44


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [8]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [9]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,safebrowsing.clients.google.com,safebrowsing-cache.google.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54843,safebrowsing.clients.google.com,safebrowsing-cache.google.com,safebrowsing.clients.google.com,safebrowsing-cache.google.com,NaN,NaN,NaN,NaN,NaN,NaN
77292,www.apache.org,www.apache.org,download.eclipse.org,www.apache.org,www.apache.org,www.webtide.com,download.oracle.com,javadl-esd-secure.oracle.com,www.caucho.com,www.apache.org
114021,www.webtide.com,download.oracle.com,www.caucho.com,download.oracle.com,www.webtide.com,www.apache.org,public.dhe.ibm.com,www.webtide.com,www.apache.org,www.apache.org
146670,public.dhe.ibm.com,jope.ow2.org,download.oracle.com,public.dhe.ibm.com,jope.ow2.org,master.dl.sourceforge.net,www.apache.org,download.eclipse.org,www.apache.org,public.dhe.ibm.com


In [10]:
full_sites = full_sites.fillna('0').astype('str')

Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [11]:
vectorizer_title = TfidfVectorizer(min_df=5, max_df=0.4, ngram_range=(1, 3))
site_title1 = vectorizer_title.fit_transform(train_df['site1'])
site_title2 = vectorizer_title.fit_transform(train_df['site2'])
site_title3 = vectorizer_title.fit_transform(train_df['site3'])
site_title4 = vectorizer_title.fit_transform(train_df['site4'])
site_title5 = vectorizer_title.fit_transform(train_df['site5'])
site_title6 = vectorizer_title.fit_transform(train_df['site6'])
site_title7 = vectorizer_title.fit_transform(train_df['site7'])
site_title8 = vectorizer_title.fit_transform(train_df['site8'])
site_title9 = vectorizer_title.fit_transform(train_df['site9'])
site_title10 = vectorizer_title.fit_transform(train_df['site10'])

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
train_df[sites]

In [ ]:
vectorizer_title.vocabulary_

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [12]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    # Ваш код здесь
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, n_jobs=1, random_state=seed)
    logit.fit(X_train, y_train)
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    return roc_auc_score(y_valid, valid_pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [13]:
%%time
get_auc_lr_valid(X_train_sparse ,y_train)

Wall time: 7.77 s


0.9195264072196881

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [14]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [15]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse, y_train)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 10.3 s


In [16]:
test_df_pred = logit.predict_proba(X_test_sparse)[:,1]

In [17]:
write_to_submission_file(test_df_pred, 'benchmark1.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [18]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [19]:
# Ваш код здесь
new_feat_train['year_month'] = train_df['time1'].apply(lambda t: t.year * 100 + t.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda t: t.year * 100 + t.month)

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [20]:
# Ваш код здесь
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1,1))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
new_feat_train['year_month'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1,1))
new_feat_test['year_month'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1,1))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [22]:
new_feat_train.head()

,year_month
session_id,
21669,-1.744405
54843,-1.744405
77292,-1.744405
114021,-1.744405
146670,-1.744405


In [23]:
X_train_sparse = csr_matrix(hstack([X_train_sparse, 
                         new_feat_train['year_month'].values.reshape(-1, 1)]))

In [24]:
X_test_sparse = csr_matrix(hstack([X_test_sparse, 
                         new_feat_test['year_month'].values.reshape(-1, 1)]))

In [25]:
X_train_sparse.shape, X_test_sparse.shape

((253561, 48372), (82797, 48372))

In [26]:
%%time
get_auc_lr_valid(X_train_sparse, y_train)

Wall time: 7.77 s


0.9196941166738521

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [64]:
# Ваш код здесь
new_feat_train['start_hour'] = train_df['time1'].apply(lambda t: t.hour)
new_feat_train['morning'] = new_feat_train['start_hour'].apply(lambda t: 1 if((t < 12)&(t >= 6)) else 0)
new_feat_train['day'] = new_feat_train['start_hour'].apply(lambda t: 1 if((t < 18)&(t >= 12)) else 0)
new_feat_train['evening'] = new_feat_train['start_hour'].apply(lambda t: 1 if((t <= 23)&(t >= 18)) else 0)
new_feat_train['night'] = new_feat_train['start_hour'].apply(lambda t: 1 if((t < 6)&(t >= 0)) else 0)
new_feat_train['weekday'] = train_df['time1'].dt.weekday

In [65]:
new_feat_test['start_hour'] = test_df['time1'].apply(lambda t: t.hour)
new_feat_test['morning'] = new_feat_test['start_hour'].apply(lambda t: 1 if((t < 12)&(t >= 6)) else 0)
new_feat_test['day'] = new_feat_test['start_hour'].apply(lambda t: 1 if((t < 18)&(t >= 12)) else 0)
new_feat_test['evening'] = new_feat_test['start_hour'].apply(lambda t: 1 if((t <= 23)&(t >= 18)) else 0)
new_feat_test['night'] = new_feat_test['start_hour'].apply(lambda t: 1 if((t < 6)&(t >= 0)) else 0)
new_feat_test['weekday'] = test_df['time1'].dt.weekday

In [67]:
scaler.fit(new_feat_train['start_hour'].values.reshape(-1,1))
scaler.fit(new_feat_train['weekday'].values.reshape(-1,1))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [68]:
new_feat_train['start_hour'] = scaler.transform(new_feat_train['start_hour'].values.reshape(-1,1))
new_feat_test['start_hour'] = scaler.transform(new_feat_test['start_hour'].values.reshape(-1,1))
new_feat_train['weekday'] = scaler.transform(new_feat_train['weekday'].values.reshape(-1,1))
new_feat_test['weekday'] = scaler.transform(new_feat_test['weekday'].values.reshape(-1,1))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(

In [69]:
new_feat_train.head()

,year_month,start_hour,morning,day,evening,night,weekday
session_id,,,,,,,
21669,-1.744405,3.545722,1,0,0,0,1.682905
54843,-1.744405,3.545722,1,0,0,0,1.682905
77292,-1.744405,3.545722,1,0,0,0,1.682905
114021,-1.744405,3.545722,1,0,0,0,1.682905
146670,-1.744405,3.545722,1,0,0,0,1.682905


In [95]:
vectorizer_title = TfidfVectorizer(min_df=3, max_df=0.3, ngram_range=(1, 3))

In [31]:
X_train_sparse_1 = csr_matrix(hstack([X_train_sparse, 
                         new_feat_train['start_hour'].values.reshape(-1, 1)]))

In [32]:
X_test_sparse_1 = csr_matrix(hstack([X_test_sparse, 
                         new_feat_test['start_hour'].values.reshape(-1, 1)]))

In [33]:
X_train_sparse_1.shape, X_test_sparse_1.shape

((253561, 48373), (82797, 48373))

In [34]:
get_auc_lr_valid(X_train_sparse_1, y_train)

0.9579184373524429

In [35]:
X_train_sparse_2 = csr_matrix(hstack([X_train_sparse, 
                         new_feat_train['morning'].values.reshape(-1, 1)]))

In [36]:
X_test_sparse_2 = csr_matrix(hstack([X_test_sparse, 
                         new_feat_test['morning'].values.reshape(-1, 1)]))

In [37]:
X_train_sparse_2.shape, X_test_sparse_2.shape

((253561, 48373), (82797, 48373))

In [38]:
get_auc_lr_valid(X_train_sparse_2, y_train)

0.9486670947710878

In [39]:
X_train_sparse_3 = csr_matrix(hstack([X_train_sparse_1, 
                         new_feat_train['morning'].values.reshape(-1, 1)]))

In [40]:
X_test_sparse_3 = csr_matrix(hstack([X_test_sparse_1, 
                         new_feat_test['morning'].values.reshape(-1, 1)]))

In [41]:
X_train_sparse_3.shape, X_test_sparse_3.shape

((253561, 48374), (82797, 48374))

In [42]:
get_auc_lr_valid(X_train_sparse_3, y_train)

0.959119424226725

In [54]:
X_train_sparse_4 = csr_matrix(hstack([
    X_train_sparse_1,
    new_feat_train['morning'].values.reshape(-1, 1),
    new_feat_train['day'].values.reshape(-1, 1),
    new_feat_train['evening'].values.reshape(-1, 1),
    new_feat_train['night'].values.reshape(-1, 1)
]))

In [86]:
X_test_sparse_4 = csr_matrix(hstack([
    X_test_sparse_1,
    new_feat_test['morning'].values.reshape(-1, 1),
    new_feat_test['day'].values.reshape(-1, 1),
    new_feat_test['evening'].values.reshape(-1, 1),
    new_feat_test['night'].values.reshape(-1, 1)
]))

In [88]:
X_train_sparse_4.shape, X_test_sparse_4.shape

((253561, 48377), (82797, 48377))

In [55]:
get_auc_lr_valid(X_train_sparse_4, y_train)

0.9629577215183759

In [70]:
X_train_sparse_5 = csr_matrix(hstack([
    X_train_sparse_4,
    new_feat_train['weekday'].values.reshape(-1, 1),
]))

In [87]:
X_test_sparse_5 = csr_matrix(hstack([
    X_test_sparse_4,
    new_feat_test['weekday'].values.reshape(-1, 1),
]))

In [89]:
X_train_sparse_5.shape, X_test_sparse_5.shape

((253561, 48378), (82797, 48378))

In [71]:
get_auc_lr_valid(X_train_sparse_5, y_train)

0.9718308498446385

записываем результаты бенчмарка

In [90]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17, C=0.06)
logit.fit(X_train_sparse_5, y_train)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 3.68 s


In [91]:
test_df_pred = logit.predict_proba(X_test_sparse_5)[:,1]

In [92]:
write_to_submission_file(test_df_pred, 'benchmark3.csv')

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [72]:
# Ваш код здесь
get_auc_lr_valid(X_train_sparse_5, y_train)

0.9718308498446385

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [79]:
# Ваш код здесь
for i in np.logspace(-3, 1, 10):
    print('C =', i, get_auc_lr_valid(X_train_sparse_5, y_train, C=i))

C = 0.001 0.8833818498216937
C = 0.0027825594022071257 0.9416671497181999
C = 0.007742636826811269 0.9671612510551657
C = 0.021544346900318832 0.9753139533058236
C = 0.05994842503189409 0.977111357869936
C = 0.1668100537200059 0.9761236684382582
C = 0.46415888336127775 0.9739437474413364
C = 1.2915496650148828 0.9709442993277131
C = 3.593813663804626 0.9661376346657102
C = 10.0 0.9598575118740105


In [84]:
arr = {}
for i in np.arange(0.05, 0.07, 0.001):
    print('C =', i, get_auc_lr_valid(X_train_sparse_5, y_train, C=i))

C = 0.06 0.9771118107307483
C = 0.061 0.9771170941068921
C = 0.062 0.9771205660397866
C = 0.063 0.9771189055501414
C = 0.064 0.9771192074573495
C = 0.065 0.9771222265294317
C = 0.066 0.9771189055501414
C = 0.067 0.9771238870190769
C = 0.068 0.9771213208078071
C = 0.069 0.9771176979213085
C = 0.07 0.9771018477928771


In [82]:
get_auc_lr_valid(X_train_sparse_5, y_train, C=0.07)

0.9771018477928771

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [ ]:
# Ваш код здесь